## Imports

In [16]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd

## PD Set Options

In [31]:
pd.set_option('display.max_columns',None)

## Data Read-in

In [17]:
df = pd.read_csv('redfin_2023-10-04-10-42-55.csv')
PBC_gf = gpd.read_file('Palm_Beach_County_Boundary.geojson')

## Data Clean

In [18]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [19]:
df = df.dropna(subset=['SOLD DATE'])

In [20]:
# Define list of desired months (excluding current month)
desired_months = ['September']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [21]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    644
Name: count, dtype: int64
-------
$/SQUARE FEET
False    643
True       1
Name: count, dtype: int64
-------
YEAR BUILT
False    644
Name: count, dtype: int64
-------


In [22]:
sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
second_newest_building = sorted_df.iloc[2]
print(second_newest_building['URL'])

https://www.redfin.com/FL/West-Palm-Beach/200-Arkona-Ct-33401/unit-2302/home/187664648


In [23]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [24]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [34]:
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
14,PAST SALE,September-20-2023,Condo/Co-op,1515 N Flagler Dr #703,West Palm Beach,FL,33401.0,1130000.0,3.0,3.0,NaN,161206.0,93013.0,1998.0,NaN,7.0,1000.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1515...,Beaches MLS,RX-10920977,N,Y,26.726457,-80.051120
121,PAST SALE,September-29-2023,Condo/Co-op,7835 Lakeside Blvd #923,Boca Raton,FL,33434.0,75000.0,2.0,2.0,Lakewood Midrise Condo,1050.0,NaN,1976.0,NaN,71.0,622.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/7835-Lake...,Beaches MLS,RX-10908172,N,Y,26.379621,-80.167448
205,PAST SALE,September-21-2023,Condo/Co-op,7786 Lakeside Blvd #623,Boca Raton,FL,33434.0,80000.0,2.0,2.0,Lakewood,1050.0,NaN,1976.0,NaN,76.0,622.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/7786-Lake...,Beaches MLS,RX-10897030,N,Y,26.378671,-80.167671
27,PAST SALE,September-28-2023,Condo/Co-op,2640 Garden Dr S #104,Lake Worth,FL,33461.0,71500.0,2.0,1.0,Lake Clarke Gardens Condo 17,894.0,NaN,1969.0,NaN,80.0,840.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth-Beach/264...,Beaches MLS,RX-10912312,N,Y,26.632381,-80.082695
248,PAST SALE,September-26-2023,Condo/Co-op,2769 S Garden Dr #206,Lake Worth,FL,33461.0,70000.0,1.0,1.0,Lake Clarke Gardens Condo 26,744.0,NaN,1971.0,NaN,94.0,617.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth-Beach/276...,Beaches MLS,RX-10911185,N,Y,26.633512,-80.084528
314,PAST SALE,September-15-2023,Condo/Co-op,6875 Willow Wood Dr #2023,Boca Raton,FL,33434.0,100000.0,2.0,2.0,Willow Wood Midrise I,1050.0,NaN,1980.0,NaN,95.0,634.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/6875-Will...,Beaches MLS,RX-10876073,N,Y,26.375667,-80.150429
581,PAST SALE,September-28-2023,Condo/Co-op,711 Lori Dr #406,Palm Springs,FL,33461.0,75000.0,1.0,1.5,Lakeside Village Condo 10,790.0,NaN,1974.0,NaN,95.0,495.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Springs/711-Lor...,Beaches MLS,RX-10879515,N,Y,26.647202,-80.093934
241,PAST SALE,September-12-2023,Condo/Co-op,2687 N Garden Dr #107,Lake Worth,FL,33461.0,77500.0,1.0,1.0,Lake Clarke Gardens Condo 5,744.0,NaN,1968.0,NaN,104.0,693.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth/2687-Gard...,Beaches MLS,RX-10901173,N,Y,26.635335,-80.083219
448,PAST SALE,September-15-2023,Condo/Co-op,8 Westgate Ln Unit 8c,Boynton Beach,FL,33436.0,186500.0,3.0,2.0,Westgate AT Hunters Run Condo,1778.0,NaN,1981.0,NaN,105.0,1052.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/8-West...,Beaches MLS,RX-10894487,N,Y,26.486472,-80.111977
323,PAST SALE,September-22-2023,Condo/Co-op,2615 NE 3rd Ct #407,Boynton Beach,FL,33435.0,80000.0,1.0,1.5,Village Royale,758.0,NaN,1974.0,NaN,106.0,403.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/2615-N...,Beaches MLS,F10391547,N,Y,26.553205,-80.061394


In [37]:
print(df_filtered['URL'].iloc[14])

https://www.redfin.com/FL/West-Palm-Beach/1515-N-Flagler-Dr-33401/unit-703/home/187603471


In [95]:
# # Correct the prices, if needed
# df_filtered.at[43,'$/SQUARE FEET']=(275000/705)

In [38]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [97]:
# # # Corrections, if needed
# df_filtered.at[245,'$/SQUARE FEET']=(210000/480)
# df_filtered.at[383,'$/SQUARE FEET']=(225000/460)
# df_filtered.at[673,'$/SQUARE FEET']=(550000/960)
# df_filtered.at[777,'$/SQUARE FEET']=(275000/697)

In [39]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
14,1130000.0,1515 N Flagler Dr #703,West Palm Beach,7.0
121,75000.0,7835 Lakeside Blvd #923,Boca Raton,71.0
205,80000.0,7786 Lakeside Blvd #623,Boca Raton,76.0
27,71500.0,2640 Garden Dr S #104,Lake Worth,80.0
248,70000.0,2769 S Garden Dr #206,Lake Worth,94.0
314,100000.0,6875 Willow Wood Dr #2023,Boca Raton,95.0
581,75000.0,711 Lori Dr #406,Palm Springs,95.0
241,77500.0,2687 N Garden Dr #107,Lake Worth,104.0
448,186500.0,8 Westgate Ln Unit 8c,Boynton Beach,105.0
323,80000.0,2615 NE 3rd Ct #407,Boynton Beach,106.0


In [99]:
print(df_filtered.URL.iloc[8])

https://www.redfin.com/FL/Boca-Raton/6845-Willow-Wood-Dr-33434/unit-3031/home/42254394


In [100]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [40]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [41]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [42]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [43]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [44]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [45]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"August 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

folium.GeoJson(PBC_gf,tooltip='Palm Beach County',name='Palm Beach County').add_to(m)

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
m

In [46]:
m.save('index.html')

## Summary Info

In [47]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [48]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [49]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [50]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Lake-Worth-Beach/2769-Garden-Dr-S-33461/unit-206/home/42285227


In [51]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
La Clara, 200 Arkona Ct #2204 West Palm Beach | Price $8,370,000 | $2,705 psf | Year built: 2023
Least Expensive
Lake Clarke Gardens Condo 26, 2769 S Garden Dr #206 Lake Worth | Price $70,000 | $94 psf | Year built: 1971
Highest Price Per Square Foot
La Clara, 200 Arkona Ct #2204 West Palm Beach | Price $8,370,000 | $2,705 psf | Year built: 2023
Lowest Price Per Square Foot
nan, 1515 N Flagler Dr #703 West Palm Beach | Price $1,130,000 | $7 psf | Year built: 1998
Newest
La Clara, 200 Arkona Ct #2204 West Palm Beach | Price $8,370,000 | $2,705 psf | Year built: 2023
Oldest
Peruvian Terrace Condo, 401 Peruvian Ave #3010 Palm Beach | Price $1,150,000 | $1,608 psf | Year built: 1963


## Time on Market Calculator

In [59]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Lake-Worth-Beach/2769-Garden-Dr-S-33461/unit-206/home/42285227


In [60]:
from datetime import datetime, timedelta

date1 = datetime(2023, 8, 11) ## List (Earlier) date
date2 = datetime(2023, 9, 26) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

46


## Map URL Snagger

In [54]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [55]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PBC_condo_sales_month_ending_sep_2023


## Get Summary Data

In [56]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 644
--------
Total sale price: $293,209,569
--------
Median sale price: $259,000
--------
Max sale price: $8,370,000
--------
Min sale price: $70,000
------------------------------------------------
PSF INFO
Max price per square foot: $2,705
--------
Min price per square foot: $7
--------
Median price per square foot: $237
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1963.0
----------
Average building age: 1983.475155279503
